Se instala Kaggle

In [1]:
!pip install kaggle==1.5.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73076 sha256=2dea0326f6a7c954a156637ee7587d3fd2d701d7b48e1ee08df4944db1100381
  Stored in directory: /root/.cache/pip/wheels/2e/27/39/f44e52756a6407b444143f233abe9fda0e18a23e8b20e0cd1c
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.5
    Uninstalling kaggle-1.7.4.5:
      Successfully uninstalled kaggle-1.7.4.5


Es necesario ubicar el archivo de autenticación de Kaggle en la dirección luego de ser cargados a la carpeta de archivos de Colab.

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Se descarga el dataset

In [3]:
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

 74% 22.0M/29.9M [00:00<00:00, 230MB/s]
100% 29.9M/29.9M [00:00<00:00, 253MB/s]


Se descomprime el dataset

In [4]:
!unzip /content/udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip

Archive:  /content/udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [7]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 121.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


Importación de librerías

In [8]:
import pandas as pd
import numpy as np
import unicodedata
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
import sklearn.model_selection
import sklearn.metrics
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau




Se carga el archivo de para realizar inferencia de datos. Debido a que el archivo es de extensión .csv se utiliza la rutina de pandas **read_csv**

In [9]:
train_dataset = pd.read_csv('/content/train.csv')

Se crean las funciones para realizar todo el preprocesado para ingresar los datos al modelo, tanto entrenamiento como inferencia. Inicialmente, se crea la función para la estandarización de nos nombres de los departamentos, para luego pasarlos a regiones.

In [10]:
def limpiar_nombre(nombre):
    if pd.isna(nombre):
        return None

    nombre_normalizado = unicodedata.normalize('NFD', str(nombre).upper())
    nombre_limpio = "".join(c for c in nombre_normalizado if unicodedata.category(c) != 'Mn')

    # Reemplazos para estandarizar al GeoJSON

    if 'BOGOTA' in nombre_limpio:
        return 'SANTAFE DE BOGOTA D.C'

    if 'SAN ANDRES' in nombre_limpio:
        return 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA'

    if 'NARINO' in nombre_limpio:
        return 'NARIÑO'

    if 'VALLE' in nombre_limpio:
        return 'VALLE DEL CAUCA'

    if 'NORTE SANTANDER' in nombre_limpio:
        return 'NORTE DE SANTANDER'

    return nombre_limpio.strip()

In [11]:
mapa_departamento_a_region = {
    'AMAZONAS': 'Amazonía',
    'ANTIOQUIA': 'Andina',
    'ARAUCA': 'Orinoquía',
    'ATLANTICO': 'Caribe', # Limpiado de 'ATLÁNTICO'
    'SANTAFE DE BOGOTA D.C': 'Andina', # Nombre estandarizado para Bogotá
    'BOLIVAR': 'Caribe',    # Limpiado de 'BOLÍVAR'
    'BOYACA': 'Andina',     # Limpiado de 'BOYACÁ'
    'CALDAS': 'Andina',
    'CAQUETA': 'Amazonía',  # Limpiado de 'CAQUETÁ'
    'CASANARE': 'Orinoquía',
    'CAUCA': 'Pacífica', # Principalmente Pacífica, aunque tiene parte Andina
    'CESAR': 'Caribe',
    'CHOCO': 'Pacífica',    # Limpiado de 'CHOCÓ'
    'CORDOBA': 'Caribe',    # Limpiado de 'CÓRDOBA'
    'CUNDINAMARCA': 'Andina',
    'GUAINIA': 'Amazonía',  # Limpiado de 'GUAINÍA'
    'GUAVIARE': 'Amazonía',
    'HUILA': 'Andina',
    'LA GUAJIRA': 'Caribe',
    'MAGDALENA': 'Caribe',
    'META': 'Orinoquía',
    'NARIÑO': 'Pacífica', # Principalmente Pacífica, aunque tiene parte Andina
    'NORTE DE SANTANDER': 'Andina',
    'PUTUMAYO': 'Amazonía',
    'QUINDIO': 'Andina',    # Limpiado de 'QUINDÍO'
    'RISARALDA': 'Andina',
    'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA': 'Insular', # Nombre estandarizado
    'SANTANDER': 'Andina',
    'SUCRE': 'Caribe',
    'TOLIMA': 'Andina',
    'VALLE DEL CAUCA': 'Pacífica',
    'VAUPES': 'Amazonía',   # Limpiado de 'VAUPÉS'
    'VICHADA': 'Orinoquía'
}


In [12]:
def normalize_text(text):

    if not isinstance(text, str):
        return ""
    try:
        text = text.upper()
        text = ''.join(c for c in unicodedata.normalize('NFD', text)
                       if unicodedata.category(c) != 'Mn')
    except Exception: # Por si acaso hay algún valor inesperado
        return ""
    return text

Se crea el diccionario para la agrupación por facultades

In [13]:
faculty_map = {
    'EDUCACION_PEDAGOGIA': ['LICENCIATURA', 'PEDAGOGIA', 'PEDAGOGÍA', 'NORMALISTA', 'ENSEÑANZA', 'DIDACTICA', 'ETNOEDUCACION'],
    'CIENCIAS_SALUD': [
        'MEDICINA', 'ENFERMERIA', 'ODONTOLOGIA', 'VETERINARIA', 'FISIOTERAPIA', 'NUTRICION', 'DIETETICA',
        'SALUD OCUPACIONAL', 'SALUD EN EL TRABAJO', 'SEGURIDAD Y SALUD EN EL TRABAJO',
        'QUIMICA FARMACEUTICA', 'FARMACIA', 'INSTRUMENTACION QUIRURGICA', 'OPTOMETRIA',
        'TERAPIA RESPIRATORIA', 'TERAPIA OCUPACIONAL', 'BACTERIOLOGIA', 'FONOAUDIOLOGIA', 'GERONTOLOGIA',
        'SALUD PUBLICA', 'BIOANALISIS', 'EPIDEMIOLOGIA', 'SALUD',
        'TERAPIAS PSICOSOCIALES', 'TERAPIA CARDIORRESPIRATORIA', 'SALUD ORAL', 'QUIRURGICA', 'FARMAC¿UTICA'
    ],
    'INGENIERIA_TECNOLOGIA': [
        'INGENIERI', 'SISTEMAS', 'SOFTWARE', 'TELECOMUNIC', 'TECNOLOGI', 'COMPUTACION', 'INFORMATICA',
        'MECANICA', 'CIVIL', 'INDUSTRIAL', 'ELECTRONICA', 'ELECTRICA', 'QUIMICA',
        'ALIMENTOS', 'PRODUCCION', 'MECATRONICA', 'BIOMEDICA', 'CONTROL', 'PROCESOS', 'AGROINDUSTRIAL',
        'MULTIMEDIA', 'SONIDO', 'PETROLEO', 'AERONAUTICA', 'NAVAL', 'TOPOGRAFIA', 'MATERIALES', 'GEOLOGICA',
        'FORESTAL', 'ELECTROMECANICA', 'TELEMATICA', 'ENERGIA', 'AGRICOLA', 'MINAS',
        'CATASTRAL', 'AUTOMATIZACION', 'PLASTICOS', 'TEXTIL', 'MANTENIMIENTO', 'BIOQUIMICA',
        'BIOTECNOLOGICA', 'OCEANOGRAFICA', 'URBANA', 'PETROQUIMICA', 'NANOTECNOLOGIA',
        'CIENCIA DE DATOS', 'INTELIGENCIA ARTIFICIAL', 'ROBOTICA', 'DESARROLLO DE SOFTWARE',
        'CONSTRUCCION', 'CIENCIAS DE LA INFORMACION Y LA DOCUMENTACION', 'INGENIER¿A', 'URBANISMO'
    ],
    'ARTES_DISENO_COM_AUDIOVISUAL': [
        'ARTE', 'ARTES', 'DISEÑO', 'DISENO', 'MUSICA', 'ARQUITECTURA', 'COMUNICACION AUDIOVISUAL', 'AUDIOVISUAL',
        'VISUAL', 'CINE', 'TELEVISION', 'FOTOGRAFIA', 'MODA', 'VESTUARIO', 'DANZA', 'TEATRO',
        'INTERIORES', 'ESCENARIOS', 'ANIMACION', 'CULINARIA', 'GASTRONOMIA', 'REALIZACION',
        'PRODUCCION MUSICAL', 'BELLAS ARTES', 'MEDIOS AUDIOVISUALES', 'COMUNICACION GRAFICA', 'GRAFICO',
        'PUBLICIDAD', 'ESCENICAS', 'PLASTICAS', 'COREOGRAFICA', 'PATRIMONIO', 'CREACION LITERARIA',
        'INTERACTIVO', 'CROSSMEDIA', 'NARRATIVAS DIGITALES', 'COMUNICACION', 'COMUNICACI¿N', 'DISE¿¿O'
    ],
    'CIENCIAS_ECONOMICAS_ADMINISTRATIVAS_CONTABLES': [
        'ADMINISTRA', 'MERCADEO', 'FINANCIERA', 'CONTADURIA', 'ECONOMIA', 'NEGOCIOS', 'COMERCIO',
        'HOTELERIA', 'TURISMO', 'GESTION', 'LOGISTICA', 'FINANZAS', 'BANCARIA', 'EMPRESARIAL',
        'AUDITORIA', 'TRIBUTARIA', 'GERENCIA', 'COMERCIAL', 'ADUANAS', 'SEGUROS', 'ADMINSITRACION', 'ECONOM¿A', 'MARKETING',
        'MERCADOLOGIA', 'RELACIONES ECONOMICAS INTERNACIONALES'
    ],
    'CIENCIAS_SOCIALES_HUMANIDADES_DERECHO': [
        'DERECHO', 'JURISPRUDENCIA', 'PSICOLOGIA', 'CIENCIA POLITICA', 'POLITICA', 'COMUNICACION SOCIAL',
        'PERIODISMO', 'TRABAJO SOCIAL', 'SOCIOLOGIA', 'FILOSOFIA', 'HISTORIA', 'ANTROPOLOGIA', 'GEOGRAFIA',
        'RELACIONES INTERNACIONALES', 'ESTUDIOS LITERARIOS', 'LITERATURA', 'LINGUISTICA', 'FILOLOGIA',
        'ARCHIVISTICA', 'BIBLIOTECOLOGIA', 'TEOLOGIA', 'PLANEACION Y DESARROLLO SOCIAL', 'CRIMINALISTICA',
        'GOBIERNO', 'ESTUDIOS POLITICOS', 'DESARROLLO FAMILIAR', 'HUMANIDADES', 'LETRAS', 'SOCIOCULTURALES',
        'PAZ', 'CONFLICTO', 'DERECHOS HUMANOS', 'ESTUDIOS CULTURALES', 'COOPERACION INTERNACIONAL',
        'CIENCIAS RELIGIOSAS', 'RELACIONES PUBLICAS', 'COMUNICACION CORPORATIVA', 'ARQUEOLOGIA', 'CIENCIAS BIBLICAS',
        'DEPOR', 'LENGUAS', 'PSICÓLOGA'
    ],
    'CIENCIAS_AGROPECUARIAS_AMBIENTALES': [
        'AGRONOMIA', 'AGROPECUARIA', 'ZOOTECNIA', 'AGROFORESTAL', 'PESQUERA', 'ACUICULTURA', 'AGRONEGOCIOS',
        'AMBIENTAL', 'ECOLOGIA', 'RECURSOS NATURALES', 'DESARROLLO RURAL', 'AGROECOLOGICA', 'AGROINDUSTRIA'
    ],
    'CIENCIAS_EXACTAS_NATURALES': [
        'ESTADISTICA', 'BIOLOGIA', 'QUIMICA',
        'MATEMATICAS', 'FISICA', 'GEOLOGIA',
        'MICROBIOLOGIA', 'GEOCIENCIAS', 'ASTRONOMIA', 'OCEANOGRAFIA',
        'BIOQUIMICA', 'BIOTECNOLOGIA',
        'CIENCIAS NATURALES', 'CIENCIAS DE LA COMPUTACION',
        'ACTUARIA', 'GEOGRAFIA FISICA', 'GEOLOG¿A', 'MATEM'
    ],
    'DEFENSA_SEGURIDAD': [
        'CIENCIAS MILITARES', 'CIENCIAS NAVALES', 'ADMINISTRACION POLICIAL', 'INVESTIGACION CRIMINAL',
        'SEGURIDAD',
    ]
}

In [14]:
faculty_order = [
    'EDUCACION_PEDAGOGIA',
    'CIENCIAS_SALUD',
    'INGENIERIA_TECNOLOGIA',
    'ARTES_DISENO_COM_AUDIOVISUAL',
    'CIENCIAS_ECONOMICAS_ADMINISTRATIVAS_CONTABLES',
    'CIENCIAS_SOCIALES_HUMANIDADES_DERECHO',
    'CIENCIAS_AGROPECUARIAS_AMBIENTALES',
    'CIENCIAS_EXACTAS_NATURALES',
    'DEFENSA_SEGURIDAD'
]

Se crea la función para asignar las facultades

In [15]:
def get_faculty(program_name_raw):

    program_name = normalize_text(program_name_raw)

    if not program_name:
        return 'OTROS_PROGRAMAS_O_INVALIDOS'

    if 'QUIMICA FARMACEUTICA' in program_name:
        return 'CIENCIAS_SALUD'

    if 'INGENIERIA QUIMICA' in program_name:
        return 'INGENIERIA_TECNOLOGIA'

    health_safety_phrases = [
        'SEGURIDAD Y SALUD EN EL TRABAJO', 'SALUD OCUPACIONAL', 'ADMINISTRACION EN SALUD OCUPACIONAL',
        'ADMINISTRACION DE LA SEGURIDAD Y SALUD OCUPACIONAL', 'ADMINISTRACION DE LA SEGURIDAD SOCIAL',
        'GESTION DE LA SEGURIDAD Y LA SALUD LABORAL', 'RIESGOS DE SEGURIDAD Y SALUD EN EL TRABAJO',
        'HIGIENE Y SEGURIDAD OCUPACIONAL', 'SEGURIDAD Y SALUD PARA EL TRABAJO',
        'SEGURIDAD INDUSTRIAL E HIGIENE OCUPACIONAL', 'SALUD LABORAL'
    ]

    for phrase in health_safety_phrases:
        if phrase in program_name:
            return 'CIENCIAS_SALUD'

    if 'CONSTRUCCION' in program_name:

        if 'ARQUITECTURA' in program_name or 'ARQUITECTONICA' in program_name:

            return 'ARTES_DISENO_COM_AUDIOVISUAL'
        else:
            return 'INGENIERIA_TECNOLOGIA'

    if 'CIENCIAS DE LA COMPUTACION' in program_name:

        if 'INTELIGENCIA ARTIFICIAL' in program_name or 'INGENIERIA' in program_name:

            return 'INGENIERIA_TECNOLOGIA'

        else:

            return 'INGENIERIA_TECNOLOGIA'

    if 'PUBLICIDAD' in program_name:

        if 'MERCADEO' in program_name or 'VENTAS' in program_name or 'COMERCIAL' in program_name or 'NEGOCIOS' in program_name or 'MARKETING' in program_name:

            return 'CIENCIAS_ECONOMICAS_ADMINISTRATIVAS_CONTABLES'

    # --- Bucle General por Palabras Clave ---
    for faculty_name in faculty_order:
        keywords = faculty_map[faculty_name]
        for keyword in keywords:
            if keyword in program_name:
                if faculty_name == 'INGENIERIA_TECNOLOGIA':
                    if keyword == 'QUIMICA' and 'FARMACEUTICA' in program_name: continue
                elif faculty_name == 'CIENCIAS_EXACTAS_NATURALES':
                    if keyword == 'QUIMICA' and ('FARMACEUTICA' in program_name or 'INGENIERI' in program_name): continue
                    if keyword == 'GEOLOGIA' and 'INGENIERI' in program_name: continue
                    if (keyword == 'BIOQUIMICA' or keyword == 'BIOTECNOLOGIA') and 'INGENIERI' in program_name: continue
                    if keyword == 'CIENCIAS DE LA COMPUTACION' and ('INGENIERI' in program_name or 'INTELIGENCIA ARTIFICIAL' in program_name): continue
                elif faculty_name == 'CIENCIAS_AGROPECUARIAS_AMBIENTALES':
                    if (keyword == 'AGROINDUSTRIAL' or keyword == 'AGROINDUSTRIA') and 'INGENIERI' in program_name: continue
                if keyword == 'SALUD' and 'ADMINISTRA' in program_name and faculty_name == 'CIENCIAS_SALUD':
                    return 'CIENCIAS_SALUD'

                return faculty_name

    return 'OTROS_PROGRAMAS_NO_CLASIFICADOS' # Para programas que no coincidan con ninguna palabra clave

In [16]:
def preprocess_data(df, train=True):

  # Eliminación de columnas innecesarias

  ids = df.pop('ID') # Se conserva ID para el caso de los datos de test

  df.drop(['PERIODO', 'coef_4'],
          axis=1,
          inplace=True)

  # Llenado de datos faltantes

  df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].fillna('Sin Estrato')
  df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('No aplica')

  # Se asignan las categorías posibles para la variable ESTU_VALORMATRICULAUNIVERSIDAD
  orden_matricula = [
    'No pagó matrícula',
    'Menos de 500 mil',
    'Entre 500 mil y menos de 1 millón',
    'Entre 1 millón y menos de 2.5 millones',
    'Entre 2.5 millones y menos de 4 millones',
    'Entre 4 millones y menos de 5.5 millones',
    'Entre 5.5 millones y menos de 7 millones',
    'Más de 7 millones',
    'No aplica'
  ]

  df['ESTU_VALORMATRICULAUNIVERSIDAD'] = pd.Categorical(
    df['ESTU_VALORMATRICULAUNIVERSIDAD'],
    categories=orden_matricula,
    ordered=True
  )

  # Preprocesamiento para extraer la región

  df['ESTU_PRGM_DEPARTAMENTO'] = df['ESTU_PRGM_DEPARTAMENTO'].apply(limpiar_nombre)
  df['ESTU_PRGM_REGION'] = df['ESTU_PRGM_DEPARTAMENTO'].map(mapa_departamento_a_region)
  df.drop('ESTU_PRGM_DEPARTAMENTO', inplace=True, axis=1)

  # Preprocesamiento para extraer la facultad

  df['FACULTAD'] = df['ESTU_PRGM_ACADEMICO'].apply(get_faculty)
  df.drop('ESTU_PRGM_ACADEMICO', inplace=True, axis=1)

  # Asignación de categorías de ESTU_HORASSEMANATRABAJA

  orden_horas = [
    '0',
    'Menos de 10 horas',
    'Entre 11 y 20 horas',
    'Entre 21 y 30 horas',
    'Más de 30 horas',
    'No aplica'
  ]

  df['ESTU_HORASSEMANATRABAJA'] = pd.Categorical(
    df['ESTU_HORASSEMANATRABAJA'],
    categories=orden_horas,
    ordered=True
  )

  # Llenado de datos faltantes y eliminación de caracteristicas innecesarias

  df.drop('FAMI_TIENEINTERNET.1', axis=1, inplace=True)
  df['FAMI_TIENEINTERNET'] = df['FAMI_TIENEINTERNET'].fillna('No')

  df.drop('FAMI_TIENELAVADORA', axis=1, inplace=True)

  df.drop('FAMI_TIENECOMPUTADOR', axis=1, inplace=True)

  df['FAMI_TIENEAUTOMOVIL'] = df['FAMI_TIENEAUTOMOVIL'].fillna('No')

  mapeo_reemplazo_strings = {
    'No sabe': 'No aplica',
  }

  df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].replace(mapeo_reemplazo_strings)
  df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].fillna('No aplica')

  df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].replace(mapeo_reemplazo_strings)
  df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].fillna('No aplica')

  df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].fillna('No aplica')
  df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].fillna('No')

  columnas_codificar = ['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
        'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENEAUTOMOVIL',
        'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_EDUCACIONMADRE',
        'ESTU_PRGM_REGION', 'FACULTAD'
  ]

  # One-hot encoding

  encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

  datos_para_codificar = df[columnas_codificar]
  datos_codificados_array = encoder.fit_transform(datos_para_codificar)
  nombres_nuevas_columnas = encoder.get_feature_names_out(input_features=columnas_codificar)

  df_codificado = pd.DataFrame(
    datos_codificados_array,
    columns=nombres_nuevas_columnas,
    index=df.index
  )

  df.drop(columns=columnas_codificar, inplace=True)
  df = pd.concat([df, df_codificado], axis=1)

  # Codificación ordinal

  orden_estratos = ['Sin Estrato', 'Estrato 1', 'Estrato 2', 'Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 6']

  ord_encoder = OrdinalEncoder(
    categories=[orden_estratos],
    handle_unknown='use_encoded_value',
    unknown_value=np.nan
  )

  df['FAMI_ESTRATOVIVIENDA'] = ord_encoder.fit_transform(df[['FAMI_ESTRATOVIVIENDA']])

  if train:

    columna_extraida = df.pop('RENDIMIENTO_GLOBAL')
    df['RENDIMIENTO_GLOBAL'] = columna_extraida


    orden_rendimiento = [
        'alto',
        'medio-alto',
        'medio-bajo',
        'bajo'
    ]


    ord_encoder_1 = OrdinalEncoder(
      categories=[orden_rendimiento],
      handle_unknown='use_encoded_value',
      unknown_value=np.nan
    )


    df['RENDIMIENTO_GLOBAL'] = ord_encoder_1.fit_transform(df[['RENDIMIENTO_GLOBAL']])

    return df, ord_encoder_1

  else:

    return ids, df

Se cargan los datasets

In [17]:
train_dataset = pd.read_csv('/content/train.csv')
test_dataset = pd.read_csv('/content/test.csv')

In [18]:
data, target_encoder = preprocess_data(train_dataset)
X = data.drop('RENDIMIENTO_GLOBAL', axis=1)
y = data['RENDIMIENTO_GLOBAL']

Se define los parámetros de la malla y se realiza el entrenamiento

In [19]:
# Define the neural network model
model = Sequential([

    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(4, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         8,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,220 (67.27 KB)

 Trainable params: 17,220 (67.27 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# Define the learning rate scheduler
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.000001)

# Train the model with the scheduler
history = model.fit(X, y, epochs=150, batch_size=32, validation_split=0.05, callbacks=[reduce_lr])

Epoch 1/150
20559/20559 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.4039 - loss: 1.2529 - val_accuracy: 0.4054 - val_loss: 1.2496 - learning_rate: 0.0010
Epoch 2/150
20559/20559 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - accuracy: 0.4046 - loss: 1.2533 - val_accuracy: 0.4048 - val_loss: 1.2484 - learning_rate: 0.0010
Epoch 3/150
20559/20559 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - accuracy: 0.4048 - loss: 1.2516 - val_accuracy: 0.4051 - val_loss: 1.2493 - learning_rate: 0.0010
Epoch 4/150
20559/20559 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - accuracy: 0.4038 - loss: 1.2536 - val_accuracy: 0.4054 - val_loss: 1.2487 - learning_rate: 0.0010
Epoch 5/150
20559/20559 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.4052 - loss: 1.2528 - val_accuracy: 0.4043 - val_loss: 1.2505 - learning_rate: 0.0010
Epoch 6/150
20559/20559 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.4052 - loss: 1.2532 - val_accuracy: 0.4045 - val_loss: 1.2498 - learning_rate: 0.0010
Epoch 7/150
20559/20559 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms

Se realiza la inferencia y se crea el archivo para realizar al entrega

In [25]:
ids, test_data = preprocess_data(test_dataset, train=False)
y_pred_proba = model.predict(test_data)
y_pred = np.argmax(y_pred_proba, axis=1)
y_pred = target_encoder.inverse_transform(y_pred.reshape(-1, 1))
y_pred = pd.DataFrame(y_pred, columns=['RENDIMIENTO_GLOBAL'])
submit = pd.concat([ids, y_pred], axis=1)
submit.to_csv('submission.csv', index=False)

9275/9275 ━━━━━━━━━━━━━━━━━━━━ 7s 800us/step
